In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install googletrans==4.0.0-rc1 gTTS datasets librosa pydub jiwer nltk audiomentations gradio

In [ ]:
!pip install --upgrade gradio==4.44.1

In [ ]:
!pip install httpx==0.13.3

In [ ]:
import whisper
from googletrans import Translator
from gtts import gTTS
import librosa
from pydub import AudioSegment
import jiwer
import nltk
import audiomentations
import gradio as gr

print("All packages imported successfully!")

In [ ]:
#testing

# import os
# os.environ["PATH"] += r";C:\ffmpeg"
# os.environ["FFMPEG_BINARY"] = r"C:\ffmpeg\ffmpeg.exe"

# import whisper
# import whisper.audio
# whisper.audio.FFMPEG_BINARY = r"C:\ffmpeg\ffmpeg.exe"

# from googletrans import Translator
# from gtts import gTTS
# import IPython.display as ipd

# # Available language codes for each service:
# #
# # Whisper (Transcription):
# #   English: 'en'
# #   Spanish: 'es'
# #   French: 'fr'
# #   German: 'de'
# #   Italian: 'it'
# #   Japanese: 'ja'
# #   Korean: 'ko'
# #   Chinese: 'zh'
# #   Hindi: 'hi'
# #   Arabic: 'ar'
# #   Portuguese: 'pt'
# #   Russian: 'ru'
# #   Bengali: 'bn'
# #   Indonesian: 'id'
# #   Vietnamese: 'vi'
# #   Turkish: 'tr'
# #
# # googletrans (Translation):
# #   English: 'en'
# #   Spanish: 'es'
# #   French: 'fr'
# #   German: 'de'
# #   Italian: 'it'
# #   Japanese: 'ja'
# #   Korean: 'ko'
# #   Chinese: 'zh-cn'
# #   Hindi: 'hi'
# #   Arabic: 'ar'
# #   Portuguese: 'pt'
# #   Russian: 'ru'
# #   Bengali: 'bn'
# #   Indonesian: 'id'
# #   Vietnamese: 'vi'
# #   Turkish: 'tr'
# #
# # gTTS (Text-to-Speech):
# #   English: 'en'
# #   Spanish: 'es'
# #   French: 'fr'
# #   German: 'de'
# #   Italian: 'it'
# #   Japanese: 'ja'
# #   Korean: 'ko'
# #   Chinese: 'zh-cn'
# #   Hindi: 'hi'
# #   Arabic: 'ar'
# #   Portuguese: 'pt'
# #   Russian: 'ru'
# #   Bengali: 'bn'
# #   Indonesian: 'id'
# #   Vietnamese: 'vi'
# #   Turkish: 'tr'

# def voice_translate(input_path, src_lang='en', target_lang='es'):
#     model = whisper.load_model("base")
#     result = model.transcribe(input_path)
#     transcribed_text = result["text"]
#     print("Transcribed Text:", transcribed_text)
#     translator = Translator()
#     translation = translator.translate(transcribed_text, src=src_lang, dest=target_lang)
#     print("Translated Text:", translation.text)
#     tts = gTTS(translation.text, lang=target_lang)
#     output_audio_file = "translated_output.mp3"
#     tts.save(output_audio_file)
#     return ipd.Audio(output_audio_file)

# print(os.path.exists(r"C:\Users\aryan\Downloads\Recording.m4a"))
# voice_translate(r"C:\Users\aryan\Downloads\Recording.m4a", src_lang='en', target_lang='ja')


In [ ]:
import gradio as gr
from googletrans import Translator

def translate_text_input(text, target_lang):
    translator = Translator()
    translation = translator.translate(text, dest=target_lang)
    return translation.text

with gr.Blocks() as demo:
    gr.Markdown("# Text Translation Interface")
    text_input = gr.Textbox(label="Enter text to translate", lines=5)
    target_lang_text = gr.Dropdown(
        choices=['en', 'es', 'fr', 'de', 'it', 'ja', 'ko', 'zh-cn', 'hi', 'ar', 'pt', 'ru', 'bn', 'id', 'vi', 'tr'],
        value='es',
        label="Target Language"
    )
    text_btn = gr.Button("Translate Text")
    text_output = gr.Textbox(label="Translated Text", lines=5)
    
    text_btn.click(fn=translate_text_input, inputs=[text_input, target_lang_text], outputs=text_output)

demo.launch(share=False)

In [ ]:
import os
os.environ["PATH"] += r";C:\ffmpeg"
os.environ["FFMPEG_BINARY"] = r"C:\ffmpeg\ffmpeg.exe"

import whisper
import whisper.audio
whisper.audio.FFMPEG_BINARY = r"C:\ffmpeg\ffmpeg.exe"

from googletrans import Translator
from gtts import gTTS
import IPython.display as ipd
import gradio as gr
import uuid

def get_iso(lang_str):
    if "(" in lang_str and ")" in lang_str:
        return lang_str.split("(")[-1].split(")")[0].strip()
    return lang_str

def voice_translate(input_path, src_lang='en', target_lang='es'):
    model = whisper.load_model("base")
    result = model.transcribe(input_path)
    transcribed_text = result["text"]
    translator = Translator()
    src = get_iso(src_lang)
    tgt = get_iso(target_lang)
    translation = translator.translate(transcribed_text, src=src, dest=tgt)
    unique_id = uuid.uuid4().hex[:8]
    base_name = os.path.splitext(os.path.basename(input_path))[0]
    output_audio_file = f"{base_name}_{unique_id}_translated.mp3"
    tts = gTTS(translation.text, lang=tgt)
    tts.save(output_audio_file)
    return transcribed_text, translation.text, output_audio_file

def process_voice_single(audio, file_path, src_lang, target_lang):
    if file_path and os.path.exists(file_path):
        input_file = file_path
    elif audio and os.path.exists(audio):
        input_file = audio
    else:
        return "No valid audio file provided.", "", ""
    transcription, translation, output_audio_file = voice_translate(input_file, src_lang, target_lang)
    return transcription, translation, output_audio_file

def process_voice_batch(audio_files, src_lang, target_lang):
    results = []
    for file in audio_files:
        transcription, translation, _ = voice_translate(file, src_lang, target_lang)
        results.append(f"File: {os.path.basename(file)}\nTranscription: {transcription}\nTranslation: {translation}\n")
    return "\n\n".join(results)

def translate_text_input(text, target_lang):
    translator = Translator()
    tgt = get_iso(target_lang)
    translation = translator.translate(text, dest=tgt)
    return translation.text

languages = [
    "English (en)",
    "Spanish (es)",
    "French (fr)",
    "German (de)",
    "Italian (it)",
    "Japanese (ja)",
    "Korean (ko)",
    "Chinese (zh-cn)",
    "Hindi (hi)",
    "Arabic (ar)",
    "Portuguese (pt)",
    "Russian (ru)",
    "Bengali (bn)",
    "Indonesian (id)",
    "Vietnamese (vi)",
    "Turkish (tr)",
    "Polish (pl)",
    "Dutch (nl)",
    "Thai (th)",
    "Swedish (sv)"
]

with gr.Blocks() as demo:
    gr.Markdown("# Voice and Text Translation Interface")
    gr.Markdown("Select your input method below.")
    with gr.Tabs():
        with gr.TabItem("Voice Input (Single)"):
            gr.Markdown("Provide an audio file either by uploading or by specifying a file path from your system.")
            with gr.Row():
                audio_input = gr.Audio(label="Upload Audio", type="filepath")
            with gr.Row():
                file_path_input = gr.Textbox(label="Or enter local file path", placeholder="e.g., C:\\path\\to\\your\\audiofile.m4a")
            with gr.Row():
                src_lang_input = gr.Dropdown(choices=languages, value="English (en)", label="Source Language")
                target_lang_input = gr.Dropdown(choices=languages, value="Spanish (es)", label="Target Language")
            single_btn = gr.Button("Translate Audio")
            transcription_output = gr.Textbox(label="Transcribed Text")
            translation_output = gr.Textbox(label="Translated Text")
            audio_output = gr.Audio(label="Translated Audio")
            single_btn.click(fn=process_voice_single, inputs=[audio_input, file_path_input, src_lang_input, target_lang_input], outputs=[transcription_output, translation_output, audio_output])
        with gr.TabItem("Voice Input (Batch)"):
            gr.Markdown("Upload multiple audio files for batch processing.")
            with gr.Row():
                audio_batch_input = gr.File(file_count="multiple", label="Upload Multiple Audio Files")
            with gr.Row():
                src_lang_batch = gr.Dropdown(choices=languages, value="English (en)", label="Source Language")
                target_lang_batch = gr.Dropdown(choices=languages, value="Spanish (es)", label="Target Language")
            batch_btn = gr.Button("Translate Batch")
            batch_output = gr.Textbox(label="Batch Translation Results", lines=10)
            batch_btn.click(fn=process_voice_batch, inputs=[audio_batch_input, src_lang_batch, target_lang_batch], outputs=batch_output)
        with gr.TabItem("Text Input"):
            text_input = gr.Textbox(label="Enter text to translate", lines=5)
            target_lang_text = gr.Dropdown(choices=languages, value="Spanish (es)", label="Target Language")
            text_btn = gr.Button("Translate Text")
            text_output = gr.Textbox(label="Translated Text", lines=5)
            text_btn.click(fn=translate_text_input, inputs=[text_input, target_lang_text], outputs=text_output)
    gr.Markdown("**Instructions:**\n- **Voice Input (Single):** Upload an audio file or enter a local file path. The system will transcribe and translate the audio, then provide an output audio file for playback.\n- **Voice Input (Batch):** Upload multiple audio files. The system processes each and lists the transcription and translation results.\n- **Text Input:** Enter text directly to translate it into the target language.\n\n*Top 20 languages include: English (en), Spanish (es), French (fr), German (de), Italian (it), Japanese (ja), Korean (ko), Chinese (zh-cn), Hindi (hi), Arabic (ar), Portuguese (pt), Russian (ru), Bengali (bn), Indonesian (id), Vietnamese (vi), Turkish (tr), Polish (pl), Dutch (nl), Thai (th), and Swedish (sv).*")
demo.launch(share=False)